# Check of the NOE quantitative reliability

## Todo:
- H1: the most intense peak per spin system is HA(i)
- H2: the 2nd most intense is HA(i-1) and the 3rd is HN(i-1)
- H3: the most intense inter-residual HN peak per spin system is HN(i-1)

In [1]:
import pandas as pd
import numpy as np

from functions import *

In [2]:
pdb_ids = ['2LEA', '2K52', '2LTM', '2KD0', '2LF2', ]
pdb_id = pdb_ids[0]

## Reading the individual 3D $^{15}N$-NOESY peak lists

In [7]:
path = f'~/Sparky/Lists/{pdb_id}.list' # set correctly

# Reading the data
df = pd.read_csv(path, header=0, index_col=None, sep='\s+')
df = tidy_list(df)
df

,res,noe,N,Hn,H,height,noe_res,inter,resnum,noe_resnum
0,S1,H,116.240,8.111,8.111,1571,S1,False,1,1
1,S1,HA,116.240,8.111,4.389,1756,S1,False,1,1
2,S1,HB2,116.240,8.111,3.750,2457,S1,False,1,1
3,S1,HB3,116.240,8.111,3.750,2457,S1,False,1,1
4,S1,Y2H,116.240,8.111,8.062,554,Y2,True,1,2
...,...,...,...,...,...,...,...,...,...,...
1789,S100,H99HB3,123.154,8.119,3.245,1350,S100,False,100,100
1790,S100,H,123.154,8.119,8.119,69967,S100,False,100,100
1791,S100,HA,123.154,8.119,4.258,2956,S100,False,100,100
1792,S100,HB2,123.154,8.119,3.849,3318,S100,False,100,100


# How many $H^{i-k}_{\alpha}$s are stronger than $H^{i}_{\alpha}$?

In [4]:
df_intra = df[~df.inter & df.noe.str.contains('HA')]
df_inter = df[df.inter & df.noe.str.contains('HA')]

In [5]:
n_anomalies = get_n_anomalies(df_strong=df_intra,
                              df_weak=df_inter)

46


In [6]:
compare_strongest_noes(df_intra, df_inter)

,height_intra,height_inter,noe_resnum
resnum,,,
1,1756,0,0
2,5887,13351,1
3,11348,7299,2
4,8560,28898,3
8,14410,32052,7
...,...,...,...
92,848,631,91
92,457,631,91
93,7823,13397,92


Where is the maximum intensity of all inter-residual $H_a$ cross-peaks is higher than residue's very own $H_a$?